In [90]:
import numpy as np
from copy import deepcopy
from icecream import ic

In [91]:
NUM_KNAPSACKS = 2
NUM_ITEMS = 10
NUM_DIMENSIONS = 2

In [92]:
VALUES = np.random.randint(0, 100, size=NUM_ITEMS) ## valori degli oggetti
WEIGHTS = np.random.randint(0, 100, size=(NUM_ITEMS, NUM_DIMENSIONS)) ## pesi degli oggetti (n.b ogni peso ha più dimensioni e i constraint devono essere rispettati in ogni dimensione)
CONSTRAINTS = np.random.randint(0, 100 * NUM_ITEMS // NUM_KNAPSACKS, size= (NUM_KNAPSACKS, NUM_DIMENSIONS)) ## tante quante le dimensioni 

In [93]:
CONSTRAINTS
# ogni zaino è una riga e ogni colonna è il constraint sulla dimensione

array([[ 32, 491],
       [436,  25]])

## TEST PROBLEMS

In [94]:
# Problem 1:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 3
NUM_ITEMS = 20
NUM_DIMENSIONS = 2
VALUES = np.random.randint(0, 100, size=NUM_ITEMS) 
WEIGHTS = np.random.randint(0, 100, size=(NUM_ITEMS, NUM_DIMENSIONS)) 
CONSTRAINTS = np.random.randint(0, 100 * NUM_ITEMS // NUM_KNAPSACKS, size= (NUM_KNAPSACKS, NUM_DIMENSIONS)) 
CONSTRAINTS

array([[330, 142],
       [171, 526],
       [279, 474]])

In [95]:
# Problem 2:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 10
NUM_ITEMS = 100
NUM_DIMENSIONS = 10
VALUES = np.random.randint(0, 1000, size=NUM_ITEMS) 
WEIGHTS = np.random.randint(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS)) 
CONSTRAINTS = np.random.randint(0, 1000 * NUM_ITEMS // NUM_KNAPSACKS, size= (NUM_KNAPSACKS, NUM_DIMENSIONS)) 
CONSTRAINTS

array([[8564,    7, 6009, 3399,  910, 9508, 3538, 3768, 7667, 5576],
       [4586, 2671, 8713, 6269, 8186, 9916, 3586, 9847, 9871, 6214],
       [1847, 6370, 8307, 7875, 7821, 4894, 8313, 5309, 7078, 9364],
       [9306, 1827, 8542, 2892, 7718,  282, 4685, 3295, 7650, 3148],
       [1319, 2002,  663, 7315, 8022, 8011, 4490, 2885, 8788, 9393],
       [8842, 3674, 3849, 4747, 2107, 9793,  668, 8068, 6655, 6114],
       [1477, 2943, 4540, 6232, 9380,  453, 7805, 2885,  295,  784],
       [4135,  317, 7781, 7248, 3742, 9654,  267, 8388, 1419, 8472],
       [4640, 8552, 3387, 9192, 5987, 2780,  257, 5547, 4301, 2922],
       [4463, 4774, 8375, 7127, 8882, 4280, 1949, 2014, 9855, 2974]])

In [96]:
# Problem 3:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 100
NUM_ITEMS = 5000
NUM_DIMENSIONS = 100
VALUES = np.random.randint(0, 1000, size=NUM_ITEMS) 
WEIGHTS = np.random.randint(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS)) 
CONSTRAINTS = np.random.randint(1000 * 10, 1000 * 2 * NUM_ITEMS // NUM_KNAPSACKS, size= (NUM_KNAPSACKS, NUM_DIMENSIONS)) 
CONSTRAINTS


array([[84199, 23839, 43712, ..., 67017, 43782, 76374],
       [77574, 29421, 14305, ..., 86052, 17557, 17648],
       [25014, 51326, 96197, ..., 96617, 46880, 28135],
       ...,
       [88236, 71329, 19493, ..., 17401, 78968, 93867],
       [66547, 40635, 64727, ..., 66755, 47390, 29196],
       [27902, 39746, 40190, ..., 37231, 41954, 17517]])

SOLUTION

In [97]:
# not used
def enforce_unique_assignment(sol):
    for i in range(NUM_ITEMS):
        assigned = np.where(sol[:, i])[0]
        if len(assigned) > 1:
            keep = np.random.choice(assigned)
            sol[:, i] = False
            sol[keep, i] = True
    return sol

In [98]:
""" Check if the solution is valid
def check_validity(solution)-> bool:
    valid_per_bag = []
    for b in range(NUM_KNAPSACKS):
        # seleziona gli item presenti nello zaino b
        items_in_b = np.where(solution[b])[0]            # indici degli item presenti
        total_weight = WEIGHTS[items_in_b].sum(axis=0)   # somma per dimensione
        valid_per_bag.append(np.all(total_weight <= CONSTRAINTS[b]))

    overall_valid = all(valid_per_bag)
    return overall_valid"""
# to avoid too many loops, we can use matrix multiplication
def check_validity(solution: np.ndarray) -> bool:
    weights_per_bag = solution.astype(int) @ WEIGHTS
    return np.all(weights_per_bag <= CONSTRAINTS)


In [99]:


def tweak_non_valid_A(knapsacs:  list[set] ) -> list[set]:
    ## remove objects from all the knapsacks that are in overweight
    #new_bags = deepcopy(knapsacs) --> we can use copy, more efficient respect to deepcopy that is a slow operation in python
    new_bags = knapsacs.copy()
    #overweight knapsacks (bags)
    weights_per_bag = new_bags.astype(int).dot(WEIGHTS)
    overweight_mask = weights_per_bag > CONSTRAINTS
    overweight_bags = np.where(np.any(overweight_mask, axis=1))[0]

    """for b in overweight_bags:
    
        items_in_bag = np.where(new_bags[b])[0]
        
        if len(items_in_bag) > 0:
            item_to_remove = np.random.choice(items_in_bag)
            new_bags[b][item_to_remove] = False 
    return new_bags"""
    # continue removing items until all knapsacks are valid, it's more efficient
    for b in overweight_bags:
        while True:
            total_weight = new_bags[b].astype(int) @ WEIGHTS
            if np.all(total_weight <= CONSTRAINTS[b]):
                break
            items_in_bag = np.where(new_bags[b])[0]
            if len(items_in_bag) == 0:
                break
            item_to_remove = np.random.choice(items_in_bag)
            new_bags[b, item_to_remove] = False
    return new_bags
#changed from list[set] to np.ndarray for efficiency
def tweak_valid_A(knapsack: np.ndarray) -> np.ndarray:
    ## it estract a knapsac and an element if it is on the knapsac it swap it with another element not in the knapsac
    #new_bags = deepcopy(knapsack) --> we can use copy, more efficient respect to deepcopy that is a slow operation in python
    new_bags = knapsack.copy()
    b = np.random.randint(0, NUM_KNAPSACKS)
    item= np.random.randint(0, NUM_ITEMS)

    
    items_in_selected_bag = np.where(new_bags[b])[0]
    items_in_other_bags = np.where(np.any(new_bags[np.arange(NUM_KNAPSACKS) != b], axis=0))[0]
    
    if item in items_in_selected_bag:
        ## remove it from the selected knapsack
        new_bags[b][item] = False
        ## add the item not in another knapsack 
        other_bags = [i for i in range(NUM_KNAPSACKS) if i != b]
        new_b = np.random.choice(other_bags)
        new_bags[new_b][item] = True
    else :
        ## add it to the selected knapsack
        new_bags[b][item] = True  
        if item in items_in_other_bags:
            ## remove from the knapsack that currently  has it
            knapsack_having_i = np.where(new_bags[:, item])[0][0]
            new_bags[knapsack_having_i][item] = False
            
        elif not check_validity(new_bags) :
            ##if the item is not in any knapsack
            ## remove a random item from the selected knapsack 
            if len(items_in_selected_bag) > 0:
                item_to_remove = np.random.choice(items_in_selected_bag)
                new_bags[b][item_to_remove] = False

    return new_bags

In [100]:
def cost(solution: list[set]) -> tuple[float, float, bool]:

    weights_per_bag = solution.astype(int).dot(WEIGHTS)  
    total_overweight = weights_per_bag - CONSTRAINTS
    total_value = np.sum(solution.astype(int).dot(VALUES))
  
    valid = not np.any(total_overweight > 0)

    # penality: sum of overweights
    penalty = np.sum(np.maximum(0, total_overweight))
    # fitness
    alpha = 10
    fitness = total_value - alpha * penalty

    return fitness, penalty, valid

In [101]:
#random solution
current_solution = np.zeros((NUM_KNAPSACKS, NUM_ITEMS), dtype=bool)
for item in range(NUM_ITEMS):
    if np.random.random() < 0.5:  # probabilità di assegnare l'oggetto
        knapsack = np.random.randint(NUM_KNAPSACKS)
        current_solution[knapsack, item] = True
first_solution = deepcopy(current_solution)
first_solution


array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [102]:


(current_fitness, current_cost, current_valid) = cost(current_solution)

MAX_STEPS = 10000


for steps in range(MAX_STEPS):
    if steps % 1000==0:
         ic(steps,current_fitness)
    
    if current_valid:
        ## valid solution
        new_solution = tweak_valid_A(current_solution)
        (new_fitness, new_cost, new_valid) = cost(new_solution)
        if new_valid:
            if new_fitness>= current_fitness: 
                current_cost= new_cost
                current_fitness= new_fitness
                current_solution = new_solution
                current_valid= new_valid
                continue

    else: 
        ## invalid solution
        new_solution = tweak_non_valid_A(current_solution)
        (new_fitness, new_cost, new_valid) = cost(new_solution)

        if not new_valid: 
            
        #still invalid solution 
            if new_cost < current_cost or (new_cost== current_cost and new_fitness>=current_fitness): 
                # new solution is closer to being valid
                current_cost = new_cost
                current_solution = new_solution
                current_fitness= new_fitness
                current_valid= new_valid
                continue
            else:
                continue
        else:
            ## new solution is valid so it's better 
            current_cost = new_cost
            current_solution = new_solution
            current_fitness= new_fitness
            current_valid= new_valid
            continue

    

print(current_cost, current_fitness, current_valid)



ic| steps: 0, current_fitness: -5599795
ic| steps: 1000, current_fitness: 1108121
ic| steps: 2000, current_fitness: 1180260
ic| steps: 3000, current_fitness: 1236680
ic| steps: 4000, current_fitness: 1282524
ic| steps: 5000, current_fitness: 1327936
ic| steps: 6000, current_fitness: 1362120
ic| steps: 7000, current_fitness: 1395602
ic| steps: 8000, current_fitness: 1418947


In [ ]:
current_fitness

1453587

In [ ]:
solution=current_solution
solution

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [ ]:

check_validity(solution)

True

In [ ]:
# Check that the same object does not appear in multiple knapsacks
np.all(solution.sum(axis=0) <= 1)

True

In [ ]:
def solution_evaluation(solution):
    total_value = 0
    percentage_weight_over_constraint = np.zeros((NUM_KNAPSACKS, NUM_DIMENSIONS))

    for b in range(NUM_KNAPSACKS): 
        knapsack_weight = np.zeros(NUM_DIMENSIONS)
        knapsack_value = 0
        for i in range(NUM_ITEMS):
            if solution[b][i]:
                knapsack_value += VALUES[i]
                knapsack_weight += WEIGHTS[i]
        total_value += knapsack_value
        percentage_weight_over_constraint[b] = knapsack_weight*100/CONSTRAINTS[b]


    return percentage_weight_over_constraint, total_value

solution_evaluation(solution)

(array([[13.59655617, 31.86302302, 11.24846977, ..., 36.60211044,
         18.66986693, 17.193682  ],
        [63.31698933, 14.95676077, 11.37486367, ..., 14.28331416,
         14.61094252, 55.71919258],
        [24.79548782, 16.26602762, 84.00930851, ..., 15.67982207,
         23.13383375, 82.01268743],
        ...,
        [78.56373022, 25.13293231, 99.44802923, ..., 38.09446781,
         13.51970006, 23.78717699],
        [ 8.05922454, 13.31272238, 50.69146027, ..., 10.85654154,
         73.81414534, 36.44207551],
        [47.95553618, 17.8858687 , 13.59164339, ..., 21.5540853 ,
         39.09076608, 22.25543898]]),
 1453587)

## comment:
After testing several approaches, I found that a more "explorative" tweak (tweak valid) function with fewer constraints performs better, as it helps avoid stagnation or early stalling. 